# GeoTrellis, GIS на Scala и некоторые паттерны проекта 

# Производительность

In [4]:
/** This method will be used in the performance section
  * to produce timing results.
  */
def time(msg: String)(f: => Unit): Unit = {
    val s = System.currentTimeMillis
    f
    println(s"[$msg] Took: ${System.currentTimeMillis - s} ms")
}

### Макросы

К примеру, часто при работе с тайлами требуется обходить все пиксели, и для этого используется макрос `cfor`. Большинство важных операций над снимками сводятся к итерации по пикселям и это тратит много процессорного времени, поэтому везде используется `while` или / и `cfor`:

```scala
cfor(0)(_ < rows, _ + 1) { row =>
  cfor(0)(_ < cols, _ + 1) { col =>
    var count = 0
    var sum = 0.0
    cfor(0)(_ < layerCount, _ + 1) { i =>
      val v = rs(i).getDouble(col, row)
      if(isData(v)) {
        count += 1
        sum += v
      }
    }

    if(count > 0) {
      tile.setDouble(col, row, sum/count)
    } else {
      tile.setDouble(col, row, Double.NaN)
    }
  }
}
```
[Source](https://github.com/locationtech/geotrellis/blob/a00c35b928e96083188d91734252d66574e3b4a7/raster/src/main/scala/geotrellis/raster/mapalgebra/local/Mean.scala#L44-L62)

Приходится работать мутабельно, и делать иммутабельный API.

In [12]:
val len = 1000000000
time("Using foreach over a range") {
    var x = 0
    for(i <- 0 until len) {
        x += i
    }
}

time("using a while loop") {
    var x = 0
    var i = 0
    while(i < len) {
        x += i
        i += 1
    }
}

import spire.syntax.cfor._

time("spire's cfor macro") {
    var x = 0
    cfor(0)(_ < len, _ + 1) { i =>
        x += i
    }
}

[Using foreach over a range] Took: 3468 ms
[using a while loop] Took: 433 ms
[spire's cfor macro] Took: 422 ms


In [6]:
def tileMeanScope = {
    import geotrellis.raster._
    
    val tiles: Seq[Tile] = ???
    val result: Tile = tiles.localMean
}

Unit

object scala.Unit

В критически важных местах используется наибыстрейший, мутабельный, страшный код, который приходится скрывать за хорошим API.

#### Макрос для инлайна перегрузок в важных для производительности случаях сравнения

В растровых данных имеется такое понятие как NoData, которое буквально значит отсутсвие данных.
Во многих алгоритмах требуется не брать в учет значения NoData клеток (большинство), т.к. это не совсем валидные данные; и наоборот, неоторые требуют только преобразования NoData значений.
По умолчанию, `Int.MinValue` -- NoData для Int, и `Double.NaN` для Double.

```scala
val i: Int = ???
val d: Double = ???

if(i == NODATA) { /* This is NoData. NODATA is a val equal to Int.MinValue */ }
if(java.lang.Double.isNaN(d)) { /* This is NoData */ }
```

В ранних версиях было множество API проблема с проверкой данных на NoData:

```scala
val x: Double = ???
val y: Int = ???

if(x == NODATA) { /* WRONG! */ }
if(java.lang.Double.isNaN(y)) { /* WRONG! */ }
```

Или
```scala
val d: Double = Double.NaN
if(d == Double.NaN) { /* Nope, never gonna happen */ }
```

Логичное решение - это перегрузка функций (одни для Int, другие для Double). Но перегрузка не эффективна в плане производительности:

In [10]:
object NoData {
    def isNoData(v: Int): Boolean = v == Int.MinValue
    def isNoData(v: Double): Boolean = java.lang.Double.isNaN(v)
}

time("Ints: constant check") { 
    var i = 0
    while(i < 10000000) {
        if(i != Int.MinValue) { i += 1 }
    }
}

time("Ints: overloaded method") { 
    var i = 0
    while(i < 10000000) {
        if(!NoData.isNoData(i)) { i += 1 }
    }
}

time("Doubles: java.lang.Double.isNaN") {
   var i = 0.0
    while(i < 10000000.0) {
        if(!java.lang.Double.isNaN(i)) { i += 1.0 }
    }
}

time("Doubles: overloaded method") {
   var i = 0.0
    while(i < 10000000.0) {
        if(!NoData.isNoData(i)) { i += 1.0 }
    }
}

[Ints: constant check] Took: 33 ms
[Ints: overloaded method] Took: 67 ms
[Doubles: java.lang.Double.isNaN] Took: 35 ms
[Doubles: overloaded method] Took: 74 ms


Проверка против константы или прямой вызов не перегруженного метода производит горадоа меньше байткода который надо будет выполнить JVM нежели перегруженого метода, который должен сделать лукап в VTable. Макросы опять помогают, можно заинлайнить функции:

In [11]:
import geotrellis.raster._

time("Ints: constant check") { 
    var i = 0
    while(i < 10000000) {
        if(i != Int.MinValue) { i += 1 }
    }
}

time("Ints: isNoData macro") { 
    var i = 0
    while(i < 10000000) {
        if(!isNoData(i)) { i += 1 }
    }
}

time("Doubles: java.lang.Double.isNaN") {
   var i = 0.0
    while(i < 10000000.0) {
        if(!java.lang.Double.isNaN(i)) { i += 1.0 }
    }
}

time("Doubles: isNoData macro") {
   var i = 0.0
    while(i < 10000000.0) {
        if(!isNoData(i)) { i += 1.0 }
    }
}

[Ints: constant check] Took: 3 ms
[Ints: isNoData macro] Took: 3 ms
[Doubles: java.lang.Double.isNaN] Took: 35 ms
[Doubles: isNoData macro] Took: 34 ms


Макрос просто инлайнит проверку на вызывающей стороне. Если кто-то еще не трогал макросы и не знает как оно работает, то это то самое с чего можно.

In [macros/src/main/scala/geotrellis/macros/NoDataMacros.scala](https://github.com/locationtech/geotrellis/blob/a00c35b928e96083188d91734252d66574e3b4a7/macros/src/main/scala/geotrellis/macros/NoDataMacros.scala#L16-L19)
```scala
object NoDataMacros {
  // ...

  def isNoDataInt_impl(ct: Context)(i: ct.Expr[Int]): ct.Expr[Boolean] = {
    import ct.universe._
    ct.Expr(q"""$i == Int.MinValue""")
  }

  // ...
}
```

In [raster/src/main/scala/geotrellis/raster/package.scala](https://github.com/locationtech/geotrellis/blob/a00c35b928e96083188d91734252d66574e3b4a7/raster/src/main/scala/geotrellis/raster/package.scala#L162)
```scala
package object raster {
  // ...
  
  def isNoData(i: Int): Boolean = macro NoDataMacros.isNoDataInt_impl

  // ...
}
```


#### Макросы, чтобы решить проблему специализации FunctionN которая не специализируется при N > 2

Функции типа `{ x: Int => x + 1 }` _специализируются_ (нет boxing'a примитивов). Пример: вызов `def foo[T](x: T)` на типе `Double`, примтив `Double` будет "упакованы??" в тип Object перед применением к функции, и "разупакован??" внутри функции. Это вызывает серьезное падение прозиводительнсти, подробнее можно почитать [тут](https://en.wikipedia.org/wiki/Object_type_%28object-oriented_programming%29#Boxing).

Функции типа `{ (x: Int, y: Int, z: Int) => x + y + z }` не специализируются.

Для приемра, определим `map` для типа `Tile`:


In [15]:
def map(tile: Tile)(f: Int => Int): Tile = {
    val result: MutableArrayTile = ArrayTile.alloc(IntCellType, 1500, 1600)
    cfor(0)(_ < tile.rows, _  + 1) { row =>
        cfor(0)(_ < tile.cols, _ + 1) { col =>
            result.set(col, row, f(tile.get(col, row)))
        }
    }
    result
}

сравним с другой функцией которая в качестве аргмуента будет иметь функцию `(Int, Int, Int) => Int`:

In [16]:
def map2(tile: Tile)(f: (Int, Int, Int) => Int): Tile = {
    val result: MutableArrayTile = ArrayTile.alloc(IntCellType, 1500, 1600)
    cfor(0)(_ < tile.rows, _  + 1) { row =>
        cfor(0)(_ < tile.cols, _ + 1) { col =>
            result.set(col, row, f(col, row, tile.get(col, row)))
        }
    }
    result
}

`Function1` быстрее `Function3` во много раз.

In [17]:
import geotrellis.raster._

val tile = ArrayTile(Array.ofDim[Int](1500 * 1600).fill(scala.util.Random.nextInt), 1500, 1600)

time("Function1") { 
    map(tile) { z => z + 1 }
}

time("Function3") { 
    map2(tile) { (col, row, z) => col * row * z + 1 }
}

[Function1] Took: 52 ms
[Function3] Took: 726 ms


Увидеть что Function3 не специализирована можно тут:
- https://github.com/scala/scala/blob/v2.12.0/src/library/scala/Function1.scala#L32
- https://github.com/scala/scala/blob/v2.12.0/src/library/scala/Function3.scala#L16

Как вариант можно делать так:

In [19]:
trait Mapper { def apply(col: Int, row: Int, z: Int): Int }
def map3(tile: Tile)(f: Mapper): Tile = {
    val result: MutableArrayTile = ArrayTile.alloc(IntCellType, 1500, 1600)
    cfor(0)(_ < tile.rows, _  + 1) { row =>
        cfor(0)(_ < tile.cols, _ + 1) { col =>
            result.set(col, row, f(col, row, tile.get(col, row)))
        }
    }
    result
}

In [20]:
time("Function1") { 
    map(tile) { z => z + 1 }
}

time("Function3") { 
    map2(tile) { (col, row, z) => col * row * z + 1 }
}

time("Mapper Trait") { 
    map3(tile)(new Mapper { def apply(col: Int, row: Int, z: Int): Int = col * row * z + 1 })
}

[Function1] Took: 107 ms
[Function3] Took: 205 ms
[Mapper Trait] Took: 111 ms


Все потому что в слуае использования трейта Mapper мы попадаем в вызов `Function1`; мы избежали боксинга типов, но API получился никакой.

Что хочется (и как это реализовано):

In [21]:
time("Tile map with Function1") {
    tile.map { z => z + 1 }
}

time("Tile map with Function3") {
    tile.map { (col, row, z) => col * row * z + 1 }
}

[Tile map with Function1] Took: 110 ms
[Tile map with Function3] Took: 79 ms


В макросе [macros/src/main/scala/geotrellis/macros/TileMacros.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/macros/src/main/scala/geotrellis/macros/TileMacros.scala#L12-L19), определяется трейт, который определяет нужные методы:
```scala
trait MacroMappableTile[T] {
  def mapIntMapper(mapper: IntTileMapper): T
  def mapDoubleMapper(mapper: DoubleTileMapper): T
}
```

также `Mapper` , похожий на тот, что определен сверху:

```scala
trait IntTileMapper {
  def apply(col: Int, row: Int, z: Int): Int
}
```

Можно использовать макрос, который будет инлайнить колл Function3 в apply метод анонимного инстанса `IntTileMapper` ([source](https://github.com/locationtech/geotrellis/blob/v1.0.0/macros/src/main/scala/geotrellis/macros/TileMacros.scala#L34-L39)):

```scala
object TileMacros {
  def intMap_impl[T <: MacroMappableTile[T]](c: Context)(f: c.Expr[(Int, Int, Int) => Int]): c.Expr[T] = {
    import c.universe._
    val self = c.Expr[MacroMappableTile[T]](c.prefix.tree)
    val tree = q"""$self.mapIntMapper(new geotrellis.macros.IntTileMapper { def apply(col: Int, row: Int, z: Int): Int = $f(col, row, z) })"""
    new InlineUtil[c.type](c).inlineAndReset[T](tree)
  }
  
  // ...
}
```

Опять используется функциональность `spire`, эту функцию для инлайна они используют у себя.

Ссылка на нас [raster/src/main/scala/geotrellis/raster/MappableTile.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/raster/src/main/scala/geotrellis/raster/MappableTile.scala)

```scala

trait MappableTile[T <: MappableTile[T]] extends MacroMappableTile[T] {

  /**
    * Map over the tiles using a function which accepts the column,
    * row, and value at that position and returns an integer.
    */
  def map(f: (Int, Int, Int) => Int): T =
    macro TileMacros.intMap_impl[T]

  // ... 
}
```

трейт, который реализуется типом `Tile`. Т.к. `Tile` наследует MacroMappableTile, надо иметь реализации map функций для трейтов, пример реализации [raster/src/main/scala/geotrellis/raster/ArrayTile.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/raster/src/main/scala/geotrellis/raster/ArrayTile.scala#L165-L173)

```scala
trait ArrayTile extends Tile with Serializable {
  // ...
  
  def mapIntMapper(mapper: IntTileMapper): Tile = {
    val tile = ArrayTile.alloc(cellType, cols, rows)
    cfor(0)(_ < rows, _ + 1) { row =>
      cfor(0)(_ < cols, _ + 1) { col =>
        tile.set(col, row, mapper(col, row, get(col, row)))
      }
    }
    tile
  }
  
  // ...
}
```


# Организация проектов

### Проблематика

Есть какие-то основные типы, и куча операций на них. Вокруг `Tile` типа есть огромное количество операций: алгебра, стастикика, и т.п. Как все это организовать?

### Решение

#### Всунуть все в один файл

<Кривая рожа>.

#### Все распределить по object'ам

Например так:


In [30]:
import geotrellis.raster.Tile 

object LocalOperations {
    def add(tile1: Tile, tile2: Tile): Tile = ???
    def add(tile: Tile, c: Int): Tile = ???
    def subtract(tile1: Tile, tile2: Tile): Tile = ???
    def subtract(tile: Tile, c: Int): Tile = ???
    def multiply(tile1: Tile, tile2: Tile): Tile = ???
    def multiply(tile: Tile, c: Int): Tile = ???
    def divide(tile1: Tile, tile2: Tile): Tile = ???
    def divide(tile: Tile, c: Int): Tile = ???
}

def foo: Tile = {
    val tile1: Tile = ???
    val tile2: Tile = ???

    import LocalOperations._

    divide(subtract(tile1, tile2), add(tile1, tile2))
}

Unit

object scala.Unit

Решение, но не красивое с точки зрения API.
Можно использовать implicit class'ы!

In [31]:
trait Tile2 extends geotrellis.raster.Tile

object LocalOperations {
    def add(tile1: Tile2, tile2: Tile2): Tile2 = ???
    def add(tile: Tile2, c: Int): Tile2 = ???
    def subtract(tile1: Tile2, tile2: Tile2): Tile2 = ???
    def subtract(tile: Tile2, c: Int): Tile2 = ???
    def multiply(tile1: Tile2, tile2: Tile2): Tile2 = ???
    def multiply(tile: Tile2, c: Int): Tile2 = ???
    def divide(tile1: Tile2, tile2: Tile2): Tile2 = ???
    def divide(tile: Tile2, c: Int): Tile2 = ???
}


implicit class withLocalOperationMethods(val self: Tile2) {
    import LocalOperations._
    
    def +(other: Tile2): Tile2 = add(self, other)
    def +(c: Int): Tile2 = add(self, c)
    def -(other: Tile2): Tile2 = subtract(self, other)
    def -(c: Int): Tile2 = subtract(self, c)
    def *(other: Tile2): Tile2 = multiply(self, other)
    def *(c: Int): Tile2 = multiply(self, c)
    def /(other: Tile2): Tile2 = divide(self, other)
    def /(c: Int): Tile2 = divide(self, c)
}

def foo: Tile2 = {
    val tile1: Tile2 = ???
    val tile2: Tile2 = ???

   (tile1 - tile2) /  (tile1 + tile2)
}

Unit

object scala.Unit

Таких операций может быть больше (так и есть), 
можно даже дальше переорганизовать все методы таким образом, чтобы `LocalOperations` наследовал все трейты с общей функциональностью.

In [40]:
trait Tile2 extends geotrellis.raster.Tile

object LocalAdd {
    def apply(tile1: Tile2, tile2: Tile2): Tile2 = ???
    def apply(tile: Tile2, c: Int): Tile2 = ???
}

trait LocalAddMethods {
    def self: Tile2
    
    def +(other: Tile2): Tile2 = LocalAdd(self, other)
    def +(c: Int): Tile2 = LocalAdd(self, c)
}

object LocalSubtract {
    def apply(tile1: Tile2, tile2: Tile2): Tile2 = ???
    def apply(tile: Tile2, c: Int): Tile2 = ???
}

trait LocalSubtractMethods {
    def self: Tile2
    
    def -(other: Tile2): Tile2 = LocalSubtract(self, other)
    def -(c: Int): Tile2 = LocalSubtract(self, c)   
}

object LocalMultiply {
    def apply(tile1: Tile2, tile2: Tile2): Tile2 = ???
    def apply(tile: Tile2, c: Int): Tile2 = ???
}

trait LocalMultiplyMethods {
    def self: Tile2

    def *(other: Tile2): Tile2 = LocalMultiply(self, other)
    def *(c: Int): Tile2 = LocalMultiply(self, c)
}

object LocalDivide {
    def apply(tile1: Tile2, tile2: Tile2): Tile2 = ???
    def apply(tile: Tile2, c: Int): Tile2 = ???
}

trait LocalDivideMethods {
    def self: Tile2

    def /(other: Tile2): Tile2 = LocalDivide(self, other)
    def /(c: Int): Tile2 = LocalDivide(self, c)
}

object LocalOperations {
    implicit class withLocalOperationMethods(val self: Tile2)
        extends LocalAddMethods
        with LocalSubtractMethods
        with LocalMultiplyMethods
        with LocalDivideMethods
}

def foo: Tile2 = {
    import LocalOperations._

    val tile1: Tile2 = ???
    val tile2: Tile2 = ???

   (tile1 - tile2) /  (tile1 + tile2)
}

Unit

object scala.Unit

К тому же у всех методов есть self `withLocalOperationMethods`, потому можно ввести такой трейт `MethodExtensions`:

```scala
/**
  * The base-trait from which all implicit classes containing
  * extension methods are derived.
  */
trait MethodExtensions[+T] extends Serializable {
  def self: T
}
```
[Source](https://github.com/locationtech/geotrellis/blob/v1.0.0/util/src/main/scala/geotrellis/util/MethodExtensions.scala)

Получается такой паттерн:

In [ ]:
import geotrellis.util.MethodExtensions

object LocalOperations {
    object LocalAdd {
        def apply(tile1: Tile2, tile2: Tile2): Tile2 = ???
        def apply(tile: Tile2, c: Int): Tile2 = ???
    }

    trait LocalAddMethods extends MethodExtensions[Tile2] {
        def +(other: Tile2): Tile2 = LocalAdd(self, other)
        def +(c: Int): Tile2 = LocalAdd(self, c)
    }

    object LocalSubtract {
        def apply(tile1: Tile2, tile2: Tile2): Tile2 = ???
        def apply(tile: Tile2, c: Int): Tile2 = ???
    }

    trait LocalSubtractMethods extends MethodExtensions[Tile2] {   
        def -(other: Tile2): Tile2 = LocalSubtract(self, other)
        def -(c: Int): Tile2 = LocalSubtract(self, c)   
    }

    object LocalMultiply {
        def apply(tile1: Tile2, tile2: Tile2): Tile2 = ???
        def apply(tile: Tile2, c: Int): Tile2 = ???
    }

    trait LocalMultiplyMethods extends MethodExtensions[Tile2] {
        def *(other: Tile2): Tile2 = LocalMultiply(self, other)
        def *(c: Int): Tile2 = LocalMultiply(self, c)
    }

    object LocalDivide {
        def apply(tile1: Tile2, tile2: Tile2): Tile2 = ???
        def apply(tile: Tile2, c: Int): Tile2 = ???
    }

    trait LocalDivideMethods extends MethodExtensions[Tile2] {
        def /(other: Tile2): Tile2 = LocalDivide(self, other)
        def /(c: Int): Tile2 = LocalDivide(self, c)
    }

    implicit class withLocalOperationMethods(val self: Tile2) extends MethodExtensions[Tile2]
        with LocalAddMethods
        with LocalSubtractMethods
        with LocalMultiplyMethods
        with LocalDivideMethods
}

def foo: Tile2 = {
    import LocalOperations._

    val tile1: Tile2 = ???
    val tile2: Tile2 = ???

   (tile1 - tile2) /  (tile1 + tile2)
}

Unit

- Пример 1 [raster/src/main/scala/geotrellis/raster/mapalgebra/local/Add.scala](https://github.com/locationtech/geotrellis/blob/a00c35b928e96083188d91734252d66574e3b4a7/raster/src/main/scala/geotrellis/raster/mapalgebra/local/Add.scala)...
- Пример 2 [raster/src/main/scala/geotrellis/raster/mapalgebra/local/LocalMethods.scala](https://github.com/locationtech/geotrellis/blob/a00c35b928e96083188d91734252d66574e3b4a7/raster/src/main/scala/geotrellis/raster/mapalgebra/local/LocalMethods.scala#L24)...
- Пример 3 [raster/src/main/scala/geotrellis/raster/package.scala](https://github.com/locationtech/geotrellis/blob/a00c35b928e96083188d91734252d66574e3b4a7/raster/src/main/scala/geotrellis/raster/package.scala#L53-L78):

```scala
package object raster {
  // ...
  
  implicit class withTileMethods(val self: Tile) extends MethodExtensions[Tile]
      with DelayedConversionTileMethods
      with costdistance.CostDistanceMethods
      with crop.SinglebandTileCropMethods
      with equalization.SinglebandEqualizationMethods
      with hydrology.HydrologyMethods
      with mapalgebra.focal.FocalMethods
      with mapalgebra.focal.hillshade.HillshadeMethods
      with mapalgebra.local.LocalMethods
      with mapalgebra.zonal.ZonalMethods
      with mask.SinglebandTileMaskMethods
      with matching.SinglebandMatchingMethods
      with merge.SinglebandTileMergeMethods
      with prototype.SinglebandTilePrototypeMethods
      with regiongroup.RegionGroupMethods
      with render.ColorMethods
      with render.JpgRenderMethods
      with render.PngRenderMethods
      with reproject.SinglebandTileReprojectMethods
      with resample.SinglebandTileResampleMethods
      with sigmoidal.SinglebandSigmoidalMethods
      with split.SinglebandTileSplitMethods
      with summary.polygonal.PolygonalSummaryMethods
      with summary.SinglebandTileSummaryMethods
      with vectorize.VectorizeMethods
      with viewshed.ViewshedMethods
      
    // ...
}
```

`Tile` это основной тип поверх которого есть огромное количество функционала. 
И еще примеры (Mask):

- Пример 1 [spark/src/main/scala/geotrellis/spark/mask/Mask.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/spark/src/main/scala/geotrellis/spark/mask/Mask.scala)...
- Пример 2 (RDD) [spark/src/main/scala/geotrellis/spark/mask/TileRDDMaskMethods.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/spark/src/main/scala/geotrellis/spark/mask/TileRDDMaskMethods.scala)...
- Пример 3 (класс жирный имплисит) [spark/src/main/scala/geotrellis/spark/mask/Implicits.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/spark/src/main/scala/geotrellis/spark/mask/Implicits.scala)
- Который можно импортить или так `import geotrellis.spark.mask.Implicits._`, или так `geotrellis.spark._`; потому что пекедж оджект spark наследует трейт `Implicits` как видно тут [spark/src/main/scala/geotrellis/spark/package.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/spark/src/main/scala/geotrellis/spark/package.scala#L51) 

### Параметры в функция, дефолтные, перегрузки, и т.п.

### Проблематика

Иногда необходимо, чтобы было или множество перегрузок apply метода, решив таким образом проблему дефолтных параметров.
К примеру рассмотрим `S3LayerWriter`, объект, который позволяет писать RDD растров на S3. Для этого, в качестве аргумента требуется `AttributeStore` с метаданными тайлов.
Упрощенная сигнатура выглядит так:

```scala
case class AttributeStore(bucket: String, prefix: String)
```

`S3LayerWriter` так же, помимо AttributeStore, также нуждается во флагах `clobber` и `oneToOne`(переапись всего слоя, или перезапись при совпадении индекса одного из тайлов). Упрощенная сигнатура выглядеть может так:

```scala
class S3LayerWriter(attributeStore: AttributeStore, clobber: Boolean, oneToOne: Boolean) {
  def write(): Unit = println(s"S3LayerWriter $attributeStore $clobber $oneToOne")
}
```

Объект компаньон:

```scala
object S3LayerWriter {
  def apply(attributeStore: AttributeStore, clobber: Boolean, oneToOne: Boolean): S3LayerWriter =
    new S3LayerWriter(attributeStore, clobber, oneToOne)
}
```

Дефолтные параметры:

```scala
object S3LayerWriter {
  def apply(
    attributeStore: AttributeStore, 
    clobber: Boolean = true, 
    oneToOne: Boolean = false
  ): S3LayerWriter =
    new S3LayerWriter(attributeStore, clobber, oneToOne)
}
```

Немного полезных перегрузок:

```scala
object S3LayerWriter {
  def apply(
    attributeStore: AttributeStore, 
    clobber: Boolean = true, 
    oneToOne: Boolean = false
  ): S3LayerWriter =
    new S3LayerWriter(attributeStore, clobber, oneToOne)

  def apply(
    bucket: String, 
    prefix: String, 
    clobber: Boolean = true, 
    oneToOne: Boolean = false
  ): S3LayerWriter =
    apply(AttributeStore(bucket, prefix), clobber, oneToOne)
}
```

Ok, так делать нельзя; почему?

### Перегрузки и дефолтные параметры

Иногда это может скомпилироваться, однако если начать собирать это в готовый джарник, возникнут проблемы уже точно.

```console
Multiple overloaded alternatives of method define default arguments
```

Так делать [нельзя](http://stackoverflow.com/questions/4652095/why-does-the-scala-compiler-disallow-overloaded-methods-with-default-arguments). Такова особенность реализации языка.

### Решение

Можно просто делать много перегрузок:

```scala
object S3LayerWriter {
  def apply(attributeStore: AttributeStore, clobber: Boolean, oneToOne: Boolean): S3LayerWriter =
    new S3LayerWriter(attributeStore, clobber, oneToOne)

  def apply(attributeStore: AttributeStore, clobber: Boolean): S3LayerWriter =
    apply(attributeStore, true, false)

  def apply(attributeStore: AttributeStore): S3LayerWriter =
    apply(attributeStore, true)

  def apply(bucket: String, prefix: String, clobber: Boolean, oneToOne: Boolean): S3LayerWriter =
    apply(AttributeStore(bucket, prefix), clobber, oneToOne)

  def apply(bucket: String, prefix: String, clobber: Boolean): S3LayerWriter =
    apply(AttributeStore(bucket, prefix), clobber, false)

  def apply(bucket: String, prefix: String): S3LayerWriter =
    apply(AttributeStore(bucket, prefix), true, false)
}
```


Или создать класс Options:

```scala
object S3LayerWriter {
  case class Options(clobber: Boolean = true, oneToOne: Boolean = false)
  object Options {
    def DEFAULT = Options()
  }

  def apply(attributeStore: AttributeStore, options: Options): S3LayerWriter =
    new S3LayerWriter(attributeStore, clobber, oneToOne)

  def apply(attributeStore: AttributeStore): S3LayerWriter =
    apply(attributeStore, Options.DEFAULT)

  def apply(bucket: String, prefix: String, options: Options): S3LayerWriter =
    apply(AttributeStore(bucket, prefix), options)

  def apply(bucket: String, prefix: String, clobber: Boolean): S3LayerWriter =
    apply(bucket, prefix, Options.DEFAULT)
}
```

Так и сделано у нас [spark/src/main/scala/geotrellis/spark/mask/Mask.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/spark/src/main/scala/geotrellis/spark/mask/Mask.scala#L39-L47)

# Типы

__Тайп Классы__ и использование __Context Bounds__

### Context Bounds

Это синтаксический сахар, следующей конструкции:

In [32]:
def sort[T](x: Seq[T])(implicit ord: Ordering[T]): Seq[T] =
    x.sorted

=

In [34]:
def sort2[T: Ordering](x: Seq[T]): Seq[T] =
    x.sorted

Два метода выше, фактически, компилируются в одно и тоже.

In [35]:
case class Foo(x: Int)

implicit def ord: Ordering[Foo] = Ordering.by(_.x)

val foos = List(3, 1, 2).map(Foo.apply)
sort2(sort(foos) ++ foos)

List(Foo(1), Foo(1), Foo(2), Foo(2), Foo(3), Foo(3))

Пример использования 
[spark/src/main/scala/geotrellis/spark/io/LayerReader.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/spark/src/main/scala/geotrellis/spark/io/LayerReader.scala):

```scala
trait LayerReader[ID] {
    // ...
    
    def read[
        K: AvroRecordCodec: Boundable: JsonFormat: ClassTag,
        V: AvroRecordCodec: ClassTag,
        M: JsonFormat: GetComponent[?, Bounds[K]]
      ](id: ID, numPartitions: Int): RDD[(K, V)] with Metadata[M]
      
    // ...
}
```

Тоже самое, без ContextBounds
```scala
trait LayerReader[ID] {
    // ...
    
    def read[K, V, M](id: ID, numPartitions: Int)(
      implicit ev1: AvroRecordCodec[K], ev2: Boundable[K], ev3: JsFormat[K], ev4: ClassTag[K],
      ev5: AvroRecordCodec[V], ev6: ClassTag[V],
      ev7: JsonFormat[M], ev8: GetComponent[M, Bounds[K]]
    ): RDD[(K, V)] with Metadata[M]
      
    // ...
}
```

Первый вариант более читабельный, и чуть позже скажу почему

### Type Lambdas и Kind Projector

Выше был `GetComponent[?, Bounds[K]]`, в примере с использованием ContextBounds. Этот ? из KindProjector'a:

[util/src/main/scala/geotrellis/util/GetComponent.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/util/src/main/scala/geotrellis/util/GetComponent.scala)
```scala
trait GetComponent[T, C] extends Serializable {
  def get: T => C
}

object GetComponent {
  def apply[T, C](_get: T => C): GetComponent[T, C] =
    new GetComponent[T, C] {
      val get = _get
    }
}
```
`GetComponent` это линза, похожая на то, что реализовано в [Monocle](https://julien-truffaut.github.io/Monocle/). Позволяет получить объект типа `C` тз объекта типа `T`.

In [36]:
import geotrellis.util.GetComponent
import geotrellis.spark.Bounds

def foo[M, K](metadata: M)(implicit gc: GetComponent[M, Bounds[K]]): Bounds[K] = 
    gc.get(metadata)

Что можно переписать так:

In [37]:
def foo[M: ({ type B[X] = GetComponent[X, Bounds[K]] })#B, K](metadata: M): Bounds[K] = 
    implicitly[GetComponent[M, Bounds[K]]].get(metadata)

Используя KindProjector:

```scala
def foo[M: GetComponent[?, Bounds[K]], K](metadata: M): Bounds[K] = 
    implicitly[GetComponent[M, Bounds[K]]].get(metadata)
```

(не компилится Toree, т.е. не поддерживает KindProjector ):)

### Guiding Principle: Never ask for more than you need

#### or, "Don't over-concretize" - Michael Pilquist


```scala
trait LayerReader[ID] {
    // ..
    
    def read[
        K: AvroRecordCodec: Boundable: JsonFormat: ClassTag,
        V: AvroRecordCodec: ClassTag,
        M: JsonFormat: GetComponent[?, Bounds[K]]
      ](id: ID, numPartitions: Int): RDD[(K, V)] with Metadata[M]
      
    // ...
}
```

На самом деле только кажется громоздко, на самом деле наоборот, в некоторым смысле даже упрощает понимание свойств, которыми должны обладать типы.


### Method Extensions и Context Bounds

![metaism comment](http://i.imgur.com/lDcKvYv.png)

[spark/src/main/scala/geotrellis/spark/split/Split.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/spark/src/main/scala/geotrellis/spark/split/Split.scala):

```scala
object Split {
  /** Splits an RDD of tiles into tiles of size (tileCols x tileRows), and updates the ProjectedExtent component of the keys.
    */
  def apply[K: Component[?, ProjectedExtent], V <: CellGrid: (? => SplitMethods[V])](
      rdd: RDD[(K, V)], tileCols: Int, tileRows: Int
  ): RDD[(K, V)] =
    rdd
      .flatMap { case (key, tile) =>
        val splitLayout =
          TileLayout(
            math.ceil(tile.cols / tileCols.toDouble).toInt,
            math.ceil(tile.rows / tileRows.toDouble).toInt,
            tileCols,
            tileRows
          )

        if(!splitLayout.isTiled) {
          Array((key, tile))
        } else {
          val ProjectedExtent(extent, crs) = key.getComponent[ProjectedExtent]
          Raster(tile, extent).split(splitLayout, Options(extend = false, cropped = false))
            .map { raster => (key.setComponent(ProjectedExtent(raster.extent, crs)), raster.tile) }
        }
      }
}
```

[raster/src/main/scala/geotrellis/raster/split/RasterSplitMethods.scala](https://github.com/locationtech/geotrellis/blob/v1.0.0/raster/src/main/scala/geotrellis/raster/split/RasterSplitMethods.scala):

```scala
abstract class RasterSplitMethods[T <: CellGrid: (? => SplitMethods[T])] extends SplitMethods[Raster[T]] {
  def split(tileLayout: TileLayout, options: Options): Array[Raster[T]] =
    self.rasterExtent.split(tileLayout, options)
      .zip(self.tile.split(tileLayout, options))
      .map { case (re, tile) => Raster(tile, re.extent) }
}
```

[spark/src/main/scala/geotrellis/spark/filter/ToSpatial.scala](https://github.com/fosskers/geotrellis/blob/1.0/spark/src/main/scala/geotrellis/spark/filter/ToSpatial.scala):

```scala
def apply[
  K: SpatialComponent: TemporalComponent: λ[α => M[α] => Functor[M, α]]: λ[α => Component[M[α], Bounds[α]]],
  V,
  M[_]
](rdd: RDD[(K, V)] with Metadata[M[K]], instant: Long): RDD[(SpatialKey, V)] with Metadata[M[SpatialKey]] = ...
```
